<a href="https://colab.research.google.com/github/cpython-projects/python_da_06_11_25/blob/main/lesson_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Що таке API

## API (Application Programming Interface)

* Це «програма для спілкування з програмою».
* Через API можна отримувати дані від зовнішніх сервісів.
* Часто використовується формат **JSON**.

## Типи API

* **Публічні** (без авторизації): можна одразу робити запит.
* **З авторизацією**: потрібен ключ (`api_key` або `token`) — як пароль.

## Де шукати API

* На сайтах сервісів → розділ *Developers* або *API Docs*
* Приклади:

  * [CoinGecko API (без ключа)](https://www.coingecko.com/en/api/documentation)
  * [CoinMarketCap API (потрібен ключ)](https://coinmarketcap.com/api/)
  * [OpenWeatherMap](https://openweathermap.org/api)
  * [NASA API](https://api.nasa.gov/)

## JSON — формат обміну даними в API

**JSON (JavaScript Object Notation)** — це текстовий формат передачі даних, який найчастіше повертають API.

Типовий JSON виглядає так:

```json
{
  "user_id": 101,
  "name": "Anna",
  "city": "Batumi",
  "orders": [
    {
      "order_id": 5001,
      "amount": 120.5,
      "status": "completed"
    },
    {
      "order_id": 5002,
      "amount": 75.0,
      "status": "pending"
    }
  ],
  "is_active": true
}
```

### Основні типи даних у JSON:

* `string` → `"Batumi"`
* `number` → `120.5`
* `boolean` → `true / false`
* `array` → `[ {...}, {...} ]`
* `object` → `{ "key": value }`
* `null`

> **Важливо:** JSON — це **рядок тексту**, а не Python-обʼєкт.


## Серіалізація і десеріалізація (ключове поняття)

### Серіалізація

**Серіалізація** — це процес **перетворення Python-обʼєкта у JSON-рядок**, щоб:

* передати його через API
* зберегти у файл
* відправити по мережі

In [2]:
data = {
    "user_id": 101,
    "name": "Anna",
    "city": "Batumi"
}

In [4]:
data

{'user_id': 101, 'name': 'Anna', 'city': 'Batumi'}

In [3]:
import json
json_string = json.dumps(data)
json_string

'{"user_id": 101, "name": "Anna", "city": "Batumi"}'

In [5]:
type(json_string)

str


### Десеріалізація

**Десеріалізація** — це зворотній процес:
**JSON → Python-обʼєкт**

In [6]:
json_string = '{"user_id": 101, "name": "Anna", "city": "Batumi"}'

python_data = json.loads(json_string)
python_data

{'user_id': 101, 'name': 'Anna', 'city': 'Batumi'}

## Як це виглядає при роботі з API

In [7]:
import requests

url = "https://jsonplaceholder.typicode.com/users/1"
response = requests.get(url)

In [8]:
response.text


'{\n  "id": 1,\n  "name": "Leanne Graham",\n  "username": "Bret",\n  "email": "Sincere@april.biz",\n  "address": {\n    "street": "Kulas Light",\n    "suite": "Apt. 556",\n    "city": "Gwenborough",\n    "zipcode": "92998-3874",\n    "geo": {\n      "lat": "-37.3159",\n      "lng": "81.1496"\n    }\n  },\n  "phone": "1-770-736-8031 x56442",\n  "website": "hildegard.org",\n  "company": {\n    "name": "Romaguera-Crona",\n    "catchPhrase": "Multi-layered client-server neural-net",\n    "bs": "harness real-time e-markets"\n  }\n}'

In [9]:
data = response.json()
data

{'id': 1,
 'name': 'Leanne Graham',
 'username': 'Bret',
 'email': 'Sincere@april.biz',
 'address': {'street': 'Kulas Light',
  'suite': 'Apt. 556',
  'city': 'Gwenborough',
  'zipcode': '92998-3874',
  'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
 'phone': '1-770-736-8031 x56442',
 'website': 'hildegard.org',
 'company': {'name': 'Romaguera-Crona',
  'catchPhrase': 'Multi-layered client-server neural-net',
  'bs': 'harness real-time e-markets'}}

In [10]:
type(data)

dict

In [ ]:
response.text

'{\n  "id": 1,\n  "name": "Leanne Graham",\n  "username": "Bret",\n  "email": "Sincere@april.biz",\n  "address": {\n    "street": "Kulas Light",\n    "suite": "Apt. 556",\n    "city": "Gwenborough",\n    "zipcode": "92998-3874",\n    "geo": {\n      "lat": "-37.3159",\n      "lng": "81.1496"\n    }\n  },\n  "phone": "1-770-736-8031 x56442",\n  "website": "hildegard.org",\n  "company": {\n    "name": "Romaguera-Crona",\n    "catchPhrase": "Multi-layered client-server neural-net",\n    "bs": "harness real-time e-markets"\n  }\n}'

## Перетворення JSON → Pandas DataFrame

Приклад API, що повертає список обʼєктів:

In [12]:
data = [
  {"date": "2025-01-01", "revenue": 1200},
  {"date": "2025-01-02", "revenue": 1500},
  {"date": "2025-01-03", "revenue": 1100}
]

import pandas as pd

df = pd.DataFrame(data)
df

,date,revenue
0,2025-01-01,1200
1,2025-01-02,1500
2,2025-01-03,1100


In [13]:
import requests

url = "https://jsonplaceholder.typicode.com/users/"
response = requests.get(url)

In [16]:
data = response.json()

In [17]:
df = pd.DataFrame(data)
df

,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


In [18]:
df = pd.read_json(url)
df

,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


## Вкладені JSON

In [19]:
data = {
  "user_id": 101,
  "orders": [
    {"order_id": 1, "amount": 120},
    {"order_id": 2, "amount": 75}
  ]
}

df = pd.json_normalize(data, record_path="orders")
df

,order_id,amount
0,1,120
1,2,75


# Робота з публічним API (CoinGecko)

**Знайдемо потрібний endpoint**

* Документація CoinGecko: [https://www.coingecko.com/en/api/documentation](https://www.coingecko.com/en/api/documentation)
* Обираємо: `/coins/markets`

```http
GET https://api.coingecko.com/api/v3/coins/markets
?vs_currency=usd&order=market_cap_desc&per_page=10&page=1
```

**Надішлемо запит через Python**

In [20]:
import requests

url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1
}

response = requests.get(url, params=params)
print(response.status_code)
data = response.json()

200


In [21]:
print(*data, sep='\n')

{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 67244, 'market_cap': 1344086585055, 'market_cap_rank': 1, 'fully_diluted_valuation': 1344092703480, 'total_volume': 37397635769, 'high_24h': 69088, 'low_24h': 66725, 'price_change_24h': -828.888136182577, 'price_change_percentage_24h': -1.21765, 'market_cap_change_24h': -16893524867.760254, 'market_cap_change_percentage_24h': -1.24128, 'circulating_supply': 19990746.0, 'total_supply': 19990837.0, 'max_supply': 21000000.0, 'ath': 126080, 'ath_change_percentage': -46.66546, 'ath_date': '2025-10-06T18:57:42.558Z', 'atl': 67.81, 'atl_change_percentage': 99066.97338, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2026-02-17T18:14:13.021Z'}
{'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price': 1964.58

In [22]:
print(data[0])

{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 67244, 'market_cap': 1344086585055, 'market_cap_rank': 1, 'fully_diluted_valuation': 1344092703480, 'total_volume': 37397635769, 'high_24h': 69088, 'low_24h': 66725, 'price_change_24h': -828.888136182577, 'price_change_percentage_24h': -1.21765, 'market_cap_change_24h': -16893524867.760254, 'market_cap_change_percentage_24h': -1.24128, 'circulating_supply': 19990746.0, 'total_supply': 19990837.0, 'max_supply': 21000000.0, 'ath': 126080, 'ath_change_percentage': -46.66546, 'ath_date': '2025-10-06T18:57:42.558Z', 'atl': 67.81, 'atl_change_percentage': 99066.97338, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2026-02-17T18:14:13.021Z'}


**Перетворимо у DataFrame**

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df[['id', 'symbol', 'current_price', 'market_cap', 'price_change_percentage_24h']]

,id,symbol,current_price,market_cap,price_change_percentage_24h
0,bitcoin,btc,94666.000000,1891788878710,-1.91536
1,ethereum,eth,3271.380000,394972111950,-1.55699
2,tether,usdt,0.999621,186799685036,-0.00517
3,binancecoin,bnb,928.280000,126705671339,-1.09255
4,ripple,xrp,2.040000,123957906224,-2.18476
5,solana,sol,141.580000,80101905246,-1.12225
6,usd-coin,usdc,1.000000,75686837868,0.03005
7,staked-ether,steth,3272.720000,29304591657,-1.48911
8,tron,trx,0.307618,29058975028,-0.53469
9,dogecoin,doge,0.136096,22947704164,-3.82133


In [31]:
df = pd.read_json('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=500')
df

HTTPError: HTTP Error 429: Too Many Requests

# Робота з API з авторизацією (CoinMarketCap)

## Реєстрація та отримання ключа

* Зареєструйтесь на [CoinMarketCap](https://coinmarketcap.com/api/)
* Після реєстрації отримаєте `X-CMC_PRO_API_KEY`

## Приклад документації:

* Документація: [https://coinmarketcap.com/api/documentation/v1/](https://coinmarketcap.com/api/documentation/v1/)
* Endpoint: `/v1/cryptocurrency/listings/latest`

## Приклад запиту з ключем

In [36]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": "17ad748a-f734-46e1-8ae0-16044d2215b8"
}
params = {
    "start": "1",
    "limit": "10",
    "convert": "USD"
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

In [37]:
response.status_code

200

In [39]:
data.keys()

dict_keys(['status', 'data'])

In [ ]:
data.keys()

dict_keys(['status', 'data'])

## Як зрозуміти структуру JSON?

In [40]:
import json
print(json.dumps(data, indent=2))

{
  "status": {
    "timestamp": "2026-02-17T18:34:15.990Z",
    "error_code": 0,
    "error_message": null,
    "elapsed": 27,
    "credit_count": 1,
    "notice": null,
    "total_count": 8817
  },
  "data": [
    {
      "id": 1,
      "name": "Bitcoin",
      "symbol": "BTC",
      "slug": "bitcoin",
      "num_market_pairs": 12562,
      "date_added": "2010-07-13T00:00:00.000Z",
      "tags": [
        "mineable",
        "pow",
        "sha-256",
        "store-of-value",
        "state-channel",
        "coinbase-ventures-portfolio",
        "three-arrows-capital-portfolio",
        "polychain-capital-portfolio",
        "binance-labs-portfolio",
        "blockchain-capital-portfolio",
        "boostvc-portfolio",
        "cms-holdings-portfolio",
        "dcg-portfolio",
        "dragonfly-capital-portfolio",
        "electric-capital-portfolio",
        "fabric-ventures-portfolio",
        "framework-ventures-portfolio",
        "galaxy-digital-portfolio",
        "huobi-capit

## Витягуємо корисні дані у DataFrame

In [41]:
coins = data['data']  # список словників
print(type(coins), len(coins))

<class 'list'> 10


In [ ]:
coins[0]

{'id': 1,
 'name': 'Bitcoin',
 'symbol': 'BTC',
 'slug': 'bitcoin',
 'num_market_pairs': 12518,
 'date_added': '2010-07-13T00:00:00.000Z',
 'tags': ['mineable',
  'pow',
  'sha-256',
  'store-of-value',
  'state-channel',
  'coinbase-ventures-portfolio',
  'three-arrows-capital-portfolio',
  'polychain-capital-portfolio',
  'binance-labs-portfolio',
  'blockchain-capital-portfolio',
  'boostvc-portfolio',
  'cms-holdings-portfolio',
  'dcg-portfolio',
  'dragonfly-capital-portfolio',
  'electric-capital-portfolio',
  'fabric-ventures-portfolio',
  'framework-ventures-portfolio',
  'galaxy-digital-portfolio',
  'huobi-capital-portfolio',
  'alameda-research-portfolio',
  'a16z-portfolio',
  '1confirmation-portfolio',
  'winklevoss-capital-portfolio',
  'usv-portfolio',
  'placeholder-ventures-portfolio',
  'pantera-capital-portfolio',
  'multicoin-capital-portfolio',
  'paradigm-portfolio',
  'bitcoin-ecosystem',
  'layer-1',
  'ftx-bankruptcy-estate',
  '2017-2018-alt-season',
  'us-st

In [43]:
df = pd.DataFrame(coins)

df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,minted_market_cap,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
0,1,Bitcoin,BTC,bitcoin,12562,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.999084e+07,1.999084e+07,False,1.354372e+12,None,1,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 67749.6583047926, 'volume_24..."
1,1027,Ethereum,ETH,ethereum,11575,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.206924e+08,1.206924e+08,True,2.401726e+11,None,2,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 1989.9559267158131, 'volume_..."
2,825,Tether USDt,USDT,tether,171317,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.837519e+11,1.879560e+11,True,1.879001e+11,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 0.9997024636702643, 'volume_..."
3,52,XRP,XRP,xrp,1799,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,6.091732e+10,9.998572e+10,False,1.483211e+11,None,4,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 1.483422517680669, 'volume_2..."
4,1839,BNB,BNB,bnb,3135,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",1.363592e+08,1.363592e+08,1.363592e+08,False,8.452971e+10,None,5,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 619.9047269769981, 'volume_2..."
5,3408,USDC,USDC,usd-coin,37516,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,7.357969e+10,7.357969e+10,False,7.361393e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",6,6.090122e+10,6.092956e+10,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 1.0004653641135686, 'volume_..."
6,5426,Solana,SOL,solana,1089,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,5.682005e+08,6.206614e+08,True,5.269192e+10,None,7,5.252369e+08,4.459073e+10,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 84.89641377387356, 'volume_2..."
7,1958,TRON,TRX,tron,1300,2017-09-13T00:00:00.000Z,"[media, payments, tron-ecosystem, layer-1, dwf...",NaN,9.472686e+10,9.472686e+10,True,2.667316e+10,None,8,9.466789e+10,2.665656e+10,None,2026-02-17T18:33:00.000Z,"{'USD': {'price': 0.28157972232774275, 'volume..."
8,74,Dogecoin,DOGE,dogecoin,1401,2013-12-15T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.687805e+11,1.687805e+11,True,1.716126e+10,None,9,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 0.1016780329923267, 'volume_..."
9,1831,Bitcoin Cash,BCH,bitcoin-cash,1137,2017-07-23T00:00:00.000Z,"[mineable, pow, sha-256, marketplace, medium-o...",2.100000e+07,1.999617e+07,1.999617e+07,False,1.128399e+10,None,10,NaN,NaN,None,2026-02-17T18:32:00.000Z,"{'USD': {'price': 564.3075010594799, 'volume_2..."


In [46]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": "17ad748a-f734-46e1-8ae0-16044d2215b8"
}
params = {
    "start": "1",
    "limit": "100",
    "convert": "USD"
}

response = requests.get(url, headers=headers, params=params)

df = pd.json_normalize(response.json(), record_path='data')

In [47]:
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address
0,1,Bitcoin,BTC,bitcoin,12562,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.999084e+07,1.999084e+07,...,1.354552e+12,58.0350,1.422932e+12,NaN,2026-02-17T18:39:00.000Z,NaN,NaN,NaN,NaN,NaN
1,1027,Ethereum,ETH,ethereum,11575,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.206924e+08,1.206924e+08,...,2.404753e+11,10.3030,2.404753e+11,NaN,2026-02-17T18:39:00.000Z,NaN,NaN,NaN,NaN,NaN
2,825,Tether USDt,USDT,tether,171317,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.837519e+11,1.879560e+11,...,1.836741e+11,7.8694,1.878765e+11,NaN,2026-02-17T18:39:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7
3,52,XRP,XRP,xrp,1799,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,6.091732e+10,9.998572e+10,...,9.015529e+10,3.8626,1.479962e+11,NaN,2026-02-17T18:39:00.000Z,NaN,NaN,NaN,NaN,NaN
4,1839,BNB,BNB,bnb,3135,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",1.363592e+08,1.363592e+08,1.363592e+08,...,8.454568e+10,3.6223,8.454568e+10,NaN,2026-02-17T18:40:00.000Z,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3773,Artificial Superintelligence Alliance,FET,artificial-superintelligence-alliance,540,2019-03-02T00:00:00.000Z,"[cosmos-ecosystem, ai-big-data, collectibles-n...",2.719494e+09,2.280213e+09,2.714385e+09,...,3.947317e+08,0.0169,4.707763e+08,NaN,2026-02-17T18:39:00.000Z,NaN,NaN,NaN,NaN,NaN
96,5488,JUST,JST,just,217,2020-05-07T00:00:00.000Z,"[defi, tron-ecosystem, tron20-ecosystem, binan...",NaN,8.815108e+09,8.815108e+09,...,3.739464e+08,0.0160,3.739464e+08,NaN,2026-02-17T18:39:00.000Z,1958.0,Tron20,TRX,tron,TCFLL5dx5ZJdKnWuesXxi1VPwjLVmWZZy9
97,38828,Kite,KITE,kite,203,2025-10-29T11:11:32.000Z,"[ai-big-data, dapp, avalanche-ecosystem, hashk...",1.000000e+10,1.800000e+09,1.000000e+10,...,3.725408e+08,0.0160,2.069671e+09,NaN,2026-02-17T18:40:00.000Z,NaN,NaN,NaN,NaN,NaN
98,6538,Curve DAO Token,CRV,curve-dao-token,951,2020-08-14T00:00:00.000Z,"[decentralized-exchange-dex-token, defi, dao, ...",3.030303e+09,1.466029e+09,2.353349e+09,...,3.710719e+08,0.0159,7.670108e+08,NaN,2026-02-17T18:39:00.000Z,1027.0,Ethereum,ETH,ethereum,0xD533a949740bb3306d119CC777fa900bA034cd52


## Як розуміти документацію:

* `GET` — означає, що потрібно робити GET-запит
* У дужках `required` / `optional` — обов’язковий або необов’язковий параметр
* Тип даних: `string`, `int`, `float`

## Як відлагоджувати:

* `response.status_code != 200` — означає помилку (https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status)
* `response.text` — покаже текст помилки
* `try-except` — ловити помилки

## Отримати топ-10 криптовалют за ринковою капіталізацією

## Відфільтрувати монети з падінням більше ніж на 1%


## Побудувати barplot